In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset

In [11]:
df2 = pd.read_csv('Playlist_data_with_lyrics.csv')
artists = pd.read_csv('artists.csv')
df = pd.read_csv('playlist_data.csv')

In [12]:
df

,Playlist_ID,Playlist_Name,Playlist_Description,Playlist_Songs,Playlist_Artists,Playlist_Song_IDs
0,6mtYuOxzl58vSGnEDtZ9uB,Pop Hits 2000s – 2025,"old but gold, all the best throwback songs to ...","['Into You', 'Glad You Came', 'Dark Horse', 'W...","['Ariana Grande', 'The Wanted', 'Katy Perry', ...","['2meEiZKWkiN28gITzFwQo5', '1OXfWI3FQMdsKKC6lk..."
1,4Jb4PDWREzNnbZcOHPcZPy,COUNTRY HITS 2025 🔥 New Country Songs + Top Hits,best country music of 2025 most popular trendi...,"['I Had Some Help (Feat. Morgan Wallen)', ""Aus...","['Post Malone', 'Dasha', 'mgk', 'Dylan Marlowe...","['5IZXB5IKAD2qlvTPJYDCFB', '2uqYupMHANxnwgeiXT..."
2,2L2HwKRvUgBv1YetudaRI3,Pop 2000-2010 Bangers,"Some bangers, but no mash","['Whatcha Say', 'Airplanes (feat. Hayley Willi...","['Jason Derulo', 'B.o.B', 'Bruno Mars', 'Tinch...","['7xkQdy0cy5ymoWT7nedvLz', '1QnvpPFP4Q3FHbDchq..."
3,327k1ryJ7j1xD7gWKpdc6o,Popular Christian Music,NaN,"['Oh My Soul', 'Stars', 'Voice of Truth', 'You...","['Casting Crowns', 'Skillet', 'Casting Crowns'...","['3s0h3pyKFrS3XX6ZLBPx6s', '6VMT3SzIMbNoR5lsUs..."
4,4muHyvSwG1wP9sI4XihC5w,Instrumental Popular Songs 2025,"Instrumental Popular Songs of 2025, 2024, Upda...","['birds of a feather - piano instrumental', 'g...","['Chilled Pig', 'Chilled Pig', 'Chilled Pig', ...","['1bE4hvWlx8gtWIMVUptOXi', '1KYe04kDLflLdoJ9kN..."
...,...,...,...,...,...,...
2284,51e9GT8ZX12eLk3U0ZKYpP,Godzilla (Clean),(Clean) (Eminem),"['Godzilla (feat. Juice WRLD)', 'Godzilla (fea...","['Eminem', 'Eminem', 'Eminem', 'KPH']","['7kP9UmZiA6YN2S3w5XObGN', '1519ABUWhVwCx3Eyvn..."
2285,2VLi4fjkYPtJomgOoIaj0C,Old (2000's) Eminem Songs,"young eminem list (Without Me , Mockinbird , T...","['Without Me', 'Mockingbird', 'Lose Yourself',...","['Eminem', 'Eminem', 'Eminem', 'Eminem', 'Emin...","['7lQ8MOhq6IN2w8EYcFNSUk', '561jH07mF1jHuk7Kla..."
2286,2WdRzPntCC3OIrov4OJgok,all Eminem albums in order,NaN,"['Infinite', 'Public Service Announcement', 'M...","['Eminem', 'Eminem', 'Eminem', 'Eminem', 'Emin...","['01xhMrdCefADl7HSttJaf7', '3GhcNCBIEMyrcpCJ8c..."
2287,1EGau2utUnkCD50M3Iu1gT,MGK VS EMINEM,This is the beef between MGK and Eminem. The d...,"['No Reason (The Mosh Pit Song)', ""Not Alike (...","['Tech N9ne', 'Eminem', 'mgk', 'Eminem']","['0tdl6DyOBsgfGzQ0kfjvdh', '28FGV3ORH14MYORd7s..."


In [3]:
# Convert stringified lists to Python lists if needed
import ast
for col in ['Playlist_Songs', 'Playlist_Artists', 'Playlist_Song_IDs']:
    df[col] = df[col].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Build target output string: "Song1 by Artist1, Song2 by Artist2, ..."
def build_output_text(row):
    songs = row['Playlist_Songs']
    artists = row['Playlist_Artists']
    paired = [f"{s} by {a}" for s, a in zip(songs, artists)]
    return ', '.join(paired)

# Build final dataset
df['input_text'] = df['Playlist_Name']
df['target_text'] = df.apply(build_output_text, axis=1)

# Split into train and validation
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['input_text'].tolist(),
    df['target_text'].tolist(),
    test_size=0.2
)

train_data = Dataset.from_dict({'input_text': train_texts, 'label': train_labels})
val_data = Dataset.from_dict({'input_text': val_texts, 'label': val_labels})

In [4]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

# Tokenize input and label
def tokenize(batch):
    input_encodings = tokenizer(batch['input_text'], padding="max_length", truncation=True, max_length=64)
    label_encodings = tokenizer(batch['label'], padding="max_length", truncation=True, max_length=256)
    input_encodings["labels"] = label_encodings["input_ids"]
    return input_encodings

train_data = train_data.map(tokenize, batched=True)
val_data = val_data.map(tokenize, batched=True)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Map: 100%|███████████████████████████| 366/366 [00:00<00:00, 1082.65 examples/s]


In [17]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./t5_playlist_model",
    evaluation_strategy="no",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=4,
    weight_decay=0.01,
    logging_dir="./logs",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data
)

trainer.train()

TypeError: TrainingArguments.__init__() got an unexpected keyword argument 'evaluation_strategy'

In [ ]:
def generate_playlist(prompt, max_length=128):
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids
    output = model.generate(input_ids, max_length=max_length, temperature=0.8)
    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    return decoded

# Example prompt
prompt = "Road trip anthems for 2020s pop hits"
print(generate_playlist(prompt))

# GPT Model w/o Descriptions

In [13]:
import pandas as pd
from datasets import Dataset
import ast
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments
import torch

In [25]:
import pandas as pd
from datasets import Dataset
import ast

# Load and parse the dataset

# Convert stringified lists to actual lists
for col in ['Playlist_Songs', 'Playlist_Artists']:
    df[col] = df[col].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

## Training Targets 1st Iteration, 3 epoch Loss ~ 3, 5 epoch Loss ~ 2.5
# Create prompt + output text for GPT-2 training
# def format_example(row):
#     song_lines = [f"{s} - {a}" for s, a in zip(row['Playlist_Songs'], row['Playlist_Artists'])]
#     return f"### Prompt: {row['Playlist_Name']}\n### Playlist:\n" + "\n".join(song_lines)

## Training Targets 2nd Iteration, 5 epoch Loss ~ 1.3
# def format_example(row):
#     lines = [f"[SONG] {s} [ARTIST] {a}" for s, a in zip(row["Playlist_Songs"], row["Playlist_Artists"])]
#     return f"### Prompt: {row['Playlist_Name']}\n### Playlist:\n" + "\n".join(lines)

## Training targets 3rd Iteration, 5 epoch Loss ~ 
def format_example(row):
    lines = [f"[SONG] {song} [ARTIST] {artist}" for song, artist in zip(row["Playlist_Songs"], row["Playlist_Artists"])]
    playlist_body = "\n".join(lines)
    
    # Include Playlist_Description in the prompt for training only
    return (
        f"### Prompt: {row['Playlist_Name']}\n"
        f"### Description: {row['Playlist_Description']}\n"
        f"### Playlist:\n{playlist_body}"
    )

df['text'] = df.apply(format_example, axis=1)

# Convert to Hugging Face Dataset
dataset = Dataset.from_dict({"text": df['text'].tolist()})

In [26]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

tokenizer = GPT2Tokenizer.from_pretrained("gpt2-medium")
tokenizer.pad_token = tokenizer.eos_token  # GPT-2 doesn't have a pad token

model = GPT2LMHeadModel.from_pretrained("gpt2-medium")
model.resize_token_embeddings(len(tokenizer))  # In case we add special tokens

# Tokenize
def tokenize(batch):
    encodings = tokenizer(batch["text"], padding="max_length", truncation=True, max_length=512)
    encodings["labels"] = encodings["input_ids"].copy()  # 🔥 Add this line
    return encodings
    
tokenized_dataset = dataset.map(tokenize, batched=True)
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Map:   0%|          | 0/2289 [00:00<?, ? examples/s]

In [27]:
from transformers import Trainer, TrainingArguments
import torch

training_args = TrainingArguments(
    output_dir="./gpt2_playlist_model_w_Descriptions",
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_device_train_batch_size=2,
    save_steps=500,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=50,
    fp16=True if torch.cuda.is_available() else False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset
)

trainer.train()

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Step,Training Loss
50,2.354300
100,1.985500
150,1.878700
200,1.854000
250,1.772700
300,1.782800
350,1.803600
400,1.711200
450,1.676800
500,1.683000


TrainOutput(global_step=5725, training_loss=1.359067278866164, metrics={'train_runtime': 1138.0674, 'train_samples_per_second': 10.057, 'train_steps_per_second': 5.03, 'total_flos': 1.062897944887296e+16, 'train_loss': 1.359067278866164, 'epoch': 5.0})

In [28]:
def generate_playlist(prompt, max_length=200):
    input_text = f"### Prompt: {prompt}\n### Playlist:\n"
    input_ids = tokenizer.encode(input_text, return_tensors="pt")

    output = model.generate(
        input_ids = input_ids.to('cuda'),
        max_length=max_length,
        temperature=0.9,
        top_p=0.95,
        do_sample=True,
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id
    )

    result = tokenizer.decode(output[0], skip_special_tokens=True)
    return result.split("### Playlist:\n")[1].strip()

# Try it out!
prompt = "I want to dance"
print(generate_playlist(prompt))

[SONG] Viva La Vida (feat. Dua Lipa) [ARTIST] Pitbull
[SONG] Fergalicious [ARTIST] Fergie
[SONG] Only Girl (In The World) [ARTIST] Rihanna
[SONG] Whatcha Say [ARTIST] Jason Derulo
[SONG] All Star [ARTIST] The Weeknd
[SONG] Bitch Better Have My Money [ARTIST] Rihanna
[SONG] Don't Cha [ARTIST] The Pussycat Dolls
[SONG] Promiscuous [ARTIST] Nelly Furtado
[SONG] DJ Got Us Fallin' In Love (feat. Pitbull) [ARTIST] USHER
[SONG] Hot N*gga [ARTIST] Petey Pablo
[SONG] Ti


In [18]:
## Import Saved Model and use with generate_playlist function
from transformers import GPT2LMHeadModel, GPT2Tokenizer

model = GPT2LMHeadModel.from_pretrained("./gpt2_playlist_model")
tokenizer = GPT2Tokenizer.from_pretrained("./gpt2_playlist_model")

1826

# GPT Model w/ Descriptions